In [1]:
call activate tensorflow

SyntaxError: invalid syntax (<ipython-input-1-de227f45b45d>, line 1)

In [2]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, BatchNormalization, Flatten, Dense
from sklearn.utils import shuffle

train = pd.read_csv('data/training.csv')
test = pd.read_csv('data/test.csv')
lookup = pd.read_csv('data/IdLookupTable.csv')

train = train.dropna()

train['Image'] = train['Image'].apply(lambda im: np.fromstring(im, dtype=int, sep=' '))
test['Image'] = test['Image'].apply(lambda im: np.fromstring(im, dtype=int, sep=' '))

x_train = np.vstack(train['Image'].values) / 255.
print(x_train)
x_test = np.vstack(test['Image'].values) / 255.

y_train = train[train.columns[:-1]].values
# y_train = (y_train - 48) / 48
y_train = y_train.astype(np.float32)

x_train, y_train = shuffle(x_train, y_train, random_state=69)

x_train = x_train.reshape(-1, 96, 96, 1)
x_test = x_test.reshape(-1, 96, 96, 1)

featureIndex = {}
for i in range(len(train.columns)):
    featureIndex[train.columns[i]] = i


# Creates model
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding="same", activation="relu", input_shape=(96, 96, 1)))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
model.add(BatchNormalization())

model.add(Conv2D(32, (5, 5), activation="relu"))
# model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (5, 5), activation="relu"))
# model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation="relu"))
# model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(500, activation="relu"))
model.add(Dropout(0.1))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))

model.add(Dense(30))

# model.summary()
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'accuracy'])

model.fit(x_train, y_train, epochs=500, shuffle=True, validation_split=0.1)

predictions = model.predict(x_test)
ans = []
for line in lookup.iterrows():
    ans += [predictions[line[1]["ImageId"] - 1, featureIndex[line[1]["FeatureName"]]]]
submission = pd.DataFrame({"RowId": list(range(1, len(lookup) + 1)), "Location": ans})
print(submission)
submission.to_csv('data/guess.csv', index=False)


Using TensorFlow backend.
C:\Users\danie\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\danie\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\danie\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\danie\Anaconda3\envs\te

Train on 1926 samples, validate on 214 samples
Epoch 1/500
1926/1926 [==============================] - 7s 4ms/step - loss: 485.5250 - mean_absolute_error: 14.8470 - acc: 0.2529 - val_loss: 24.9184 - val_mean_absolute_error: 3.4286 - val_acc: 0.6963
Epoch 2/500
1926/1926 [==============================] - 1s 766us/step - loss: 59.6436 - mean_absolute_error: 6.0140 - acc: 0.4081 - val_loss: 16.6596 - val_mean_absolute_error: 2.8826 - val_acc: 0.6963
Epoch 3/500
1926/1926 [==============================] - 1s 775us/step - loss: 44.2523 - mean_absolute_error: 5.2257 - acc: 0.4341 - val_loss: 14.4930 - val_mean_absolute_error: 2.7855 - val_acc: 0.7009
Epoch 4/500
1926/1926 [==============================] - 2s 795us/step - loss: 38.7770 - mean_absolute_error: 4.8797 - acc: 0.4564 - val_loss: 11.4718 - val_mean_absolute_error: 2.4413 - val_acc: 0.7009
Epoch 5/500
1926/1926 [==============================] - 1s 771us/step - loss: 35.1466 - mean_absolute_error: 4.6304 - acc: 0.4969 - val_loss

KeyboardInterrupt: 